In [136]:
import json
import numpy as np
import xobjects as xo
import xtrack as xt
import xpart as xp
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy import constants 

#references: https://link.springer.com/content/pdf/10.1007/s10751-005-9217-x.pdf?pdf=button
#coasting beam: https://accelconf.web.cern.ch/cool2013/papers/tham1ha04.pdf

# Ion properties:
m_u = 931.49410242e6 # eV/c^2 -- atomic mass unit
A = 12.011 # Weight of C
Z = 6  # Number of protons in the ion (C)
Ne = 3 # Number of remaining electrons (Lithium-like)
m_e = 0.511e6 # eV/c^2 -- electron mass
m_p = 938.272088e6 # eV/c^2 -- proton mass
clight = 299792458.0 # m/s

q0=Z-Ne

mass0 = A*m_u + Ne*m_e # eV/c^2

beta_rel = 0.47
gamma_rel = 1.13

p0c = mass0*beta_rel*gamma_rel #eV/c

# p0c = equiv_proton_momentum*(Z-Ne) # eV/c
gamma = np.sqrt( 1 + (p0c/mass0)**2 ) # ion relativistic factor
beta = np.sqrt(1-1/(gamma*gamma)) # ion beta

particle_ref = xp.Particles(p0c=p0c, mass0=mass0, q0=q0, gamma=gamma)


In [137]:
p0c

5942854271.53889

In [138]:
#https://www.sciencedirect.com/science/article/pii/0168583X87905830

#twiss parameters measurments:
# https://accelconf.web.cern.ch/ipac2023/pdf/TUPM095.pdf

circumference =  108.36#m
T = circumference/(clight*beta_rel)
s_per_turn = T
turn_per_sec=1/T

beta_x = 5
beta_y = 5

disp_x = 0
qx = 2.21
qy = 2.41
dQx = 0
dQy = 0

voltage_rf = 5 *1e3 # eV
lag_rf = 0
frequency = 5*1e6 # Hz
momentum_compaction_factor = 0.02

# arc = xt.LineSegmentMap(
#         qx=Q_x, qy=Q_y,
#         length=circumference,
#         betx=beta_x,
#         bety=beta_y
#         )

alpha_x=0
alpha_y=0
Dx=0
Dpx=0
Dy=0
Dpy=0

arc = xt.LineSegmentMap(
        qx=qx, qy=qy,
        dqx=0, dqy=0,
        length=circumference,
        alfx=alpha_x,
        alfy=alpha_y,
        betx=beta_x,
        bety=beta_y,
        
        dx=Dx,
        dpx=Dpx,
        dy=Dy,
        dpy=Dpy,
        voltage_rf=voltage_rf,
        lag_rf=lag_rf,
        frequency_rf=frequency,
        momentum_compaction_factor=momentum_compaction_factor,
        longitudinal_mode = 'nonlinear'
        )


In [139]:
# emittance=10*1e-6 #inital emittance
# num_particles = int(1e3)

# sigma_x = np.sqrt(beta_x*emittance)
# sigma_px = np.sqrt(emittance*1/beta_x)
# sigma_y = np.sqrt(beta_y*emittance)
# sigma_py = np.sqrt(emittance*1/beta_y)
# sigma_p = 2e-5 # relative ion momentum spread
# sigma_p = 2e-4 # relative ion momentum spread

# delta = np.random.uniform(low=2.6e-4, high=3e-4, size=num_particles)

# #delta = np.random.normal(loc=0.0, scale=sigma_p, size=num_particles)
# x = np.random.normal(loc=0.0, scale=sigma_x, size=num_particles) + disp_x * delta
# px = np.random.normal(loc=0.0, scale=sigma_px, size=num_particles)
# y = np.random.normal(loc=0.0, scale=sigma_y, size=num_particles)
# py = np.random.normal(loc=0.0, scale=sigma_py, size=num_particles)

# particles = xp.Particles(
#     mass0=mass0,
#     p0c=p0c,
#     q0=q0,
#     x=0,
#     px=0,
#     y=0,
#     py=0,
#     delta=delta,
#     zeta=0
# )

In [140]:
num_particles = int(1e4)
bunch_intensity = int(1e6)

g_emitt = 5*1e-6

nemitt = beta * gamma * g_emitt

sigma_delta = 1e-5  # Relative energy spread

line_arc=xt.Line(
        elements=[arc])
line_arc.build_tracker()



Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [141]:
sigma_z

0.1

In [142]:
sigma_z=1e-1

particles = xp.generate_matched_gaussian_bunch(
        num_particles=num_particles,
        #total_intensity_particles=bunch_intensity,
        nemitt_x=nemitt, nemitt_y=nemitt, sigma_z=sigma_z,
        particle_ref=particle_ref,
        line=line_arc
        )

np.std(particles.delta)

LinkedArrayCpu(1.19881517e-05)

In [143]:
##################
# Laser Cooler #
##################

#laser-ion beam collision angle
theta_l = 2.6*np.pi/180 # rad
theta_l = 0
nx = 0; ny = -np.sin(theta_l); nz = -np.cos(theta_l)

# Ion excitation energy:
# hw0 = 230.823 # eV
hc=constants.hbar*clight/constants.e # eV*m (ħc)
lambda_0 = 154.82 *1e-9 # m -- ion excitation wavelength
hw0 = 2*np.pi*hc/lambda_0 # m -- ion excitation wavelength
ion_excited_lifetime=3.8e-9 #souce: Combined Laser and Electron Cooling of Bunched C3+ Ion Beams at the Storage Ring ESR

lambda_l = lambda_0*gamma_rel*(1 + beta_rel*np.cos(theta_l)) # m -- laser wavelength
#lambda_l = lambda_l*(1+sigma_delta)
lambda_l = 257.5255*1e-9
#2.57557259253e-07
# lambda_l =  2.57557259253e-07 #m #257

# Shift laser wavelength for fast longitudinal cooling:
#lambda_l = lambda_l*(1+1*sigma_p) # m

laser_frequency = clight/lambda_l # Hz


print('Laser wavelength = %.2f nm' % (lambda_l/1e-9))

laser_waist_radius = 2000
laser_intensity=50000 #k=8.6

cooling_section_length=25
GF_IP = xt.CWLaser(_buffer=buf,
                      laser_x=0,
                      laser_y=0,
                      laser_z=0,
                      
                      laser_direction_nx = 0,
                      laser_direction_ny = 0,
                      laser_direction_nz = -1,
                      laser_wavelength = lambda_l, # m
                      laser_waist_radius = laser_waist_radius, # m
                      laser_intensity=laser_intensity,
                      ion_excitation_energy = hw0, # eV
                      ion_excited_lifetime  = ion_excited_lifetime, # sec,
                      cooling_section_length=cooling_section_length,                          
   )

# simulation parameters: simulate 10 s of cooling, and take data once every 100 ms
max_time_s = 0.1
int_time_s = 0.001
T_per_turn = circumference/(clight*beta)
num_turns = int(max_time_s/T_per_turn)
save_interval = int(int_time_s/T_per_turn)


# create a monitor object, to reduce holded data
monitor = xt.ParticlesMonitor(start_at_turn=0, stop_at_turn=1,
                        n_repetitions=int(num_turns/save_interval),
                        repetition_period=save_interval,
                        num_particles=num_particles)
                        
line = xt.Line(
        elements=[monitor,GF_IP,arc])

# line = xt.Line(
#         elements=[monitor,arc])

context = xo.ContextCpu(omp_num_threads=4)
line.build_tracker(_context=context)

line.track(particles, num_turns=num_turns,
        turn_by_turn_monitor=False,with_progress=True)

# extract relevant values
x = monitor.x[:,:,0]
px = monitor.px[:,:,0]
y = monitor.y[:,:,0]
py = monitor.py[:,:,0]
delta = monitor.delta[:,:,0]
zeta = monitor.zeta[:,:,0]
state = monitor.state[:,:,0]
time = monitor.at_turn[:, 0, 0] * T_per_turn

gamma_x=(1+alpha_x**2)/beta_x
gamma_y=(1+alpha_y**2)/beta_y

action_x = (gamma_x*(x-Dx*delta)**2 + 2*alpha_x*(x-Dx*delta)*(px-Dpx*delta)+ beta_x*(px-Dpx*delta)**2)
action_y = (gamma_y*(y-Dy*delta)**2 + 2*alpha_y*(y-Dy*delta)*(py-Dpy*delta)+ beta_y*(py-Dpy*delta)**2)

emittance_x_twiss=np.mean(action_x,axis=1)*gamma/2
rms_dp_p=np.std(delta,axis=1)


np.savez(f'results/GSI.npz', x=x, px=px, y=y, py=py, zeta=zeta, delta=delta,
        action_x=action_x,action_y=action_y,emittance_x=emittance_x_twiss,state=state, time=time,s_per_turn=T_per_turn)

Laser wavelength = 257.53 nm
Compiling ContextCpu kernels...


28da822033f94b04984345220465a73d.c: In function 'CWLaser_track_local_particle':
28da822033f94b04984345220465a73d.c:4214:16: warning: unused variable 'DeltaDetuningTau' [-Wunused-variable]
 4214 |         double DeltaDetuningTau = DeltaDetuning/(2.0*gamma);
      |                ^~~~~~~~~~~~~~~~


Done compiling ContextCpu kernels.


Tracking:   0%|          | 0/129800 [00:00<?, ?it/s]

In [144]:
delta[0]

LinkedArrayCpu([-2.47045025e-06, -1.75726408e-05,  2.04168074e-05, ...,
                -7.06168995e-06, -2.36043628e-06, -1.25596505e-05])